<img src = "https://github.com/barcelonagse-datascience/academic_files/raw/master/bgsedsc_0.jpg">
$\newcommand{\bb}{\boldsymbol{\beta}}$
$\DeclareMathOperator{\Gau}{\mathcal{N}}$
$\newcommand{\bphi}{\boldsymbol \phi}$
$\newcommand{\bx}{\boldsymbol{x}}$
$\newcommand{\bu}{\boldsymbol{u}}$
$\newcommand{\by}{\boldsymbol{y}}$
$\newcommand{\whbb}{\widehat{\bb}}$
$\newcommand{\hf}{\hat{f}}$
$\newcommand{\tf}{\tilde{f}}$
$\newcommand{\ybar}{\overline{y}}$
$\newcommand{\E}{\mathbb{E}}$
$\newcommand{\Var}{Var}$
$\newcommand{\Cov}{Cov}$
$\newcommand{\Cor}{Cor}$

# Project: Linear models

## Programming project: real estate assesment evaluation

Home valuation is key in real estate industry, and also the basis for mortgages in credit sector. Here we have to predict the estimated value of a property.

Data (*Regression_Supervised_Train.csv*) consist of a list of features plus the resulting <i>parcelvalue</i>, described in *Case_data_dictionary.xlsx* file. Each row corresponds to a particular home valuation, and <i>transactiondate</i> is the date when the property was effectively sold. Properties are defined by <i>lotid</i>, but be aware that one property can be sold more than once (it's not the usual case). Also notice that some features are sometime empty, your model has to deal with it.

Note that you shouldn't use <i>totaltaxvalue</i>, <i>buildvalue</i> or <i>landvalue</i>, because they are closely correlated with the final value to predict. There is a further member of the training set predictors which is not available in the test set and therefore needs removing. 

+ Using this data build a predictive model for <b>parcelvalue</b> 
+ In your analysis for faster algorithms use the MSE criterion for choosing any hyperparameters 
+ Try a first quick implementation, then try to optimize hyperparameters
+ For this analysis there is an extra test dataset. Once your code is submitted we will run a competition to see how you score in the test data. Hence have prepared also the necessary script to compute the MSE estimate on the test data once released.
+ Bonus: Try an approach to fill NA without removing features or observations, and check improvements.

You can follow those **steps** in your first implementation:
1. *Explore* and understand the dataset. Report missing data
2. As a simplified initial version, get rid of *missing data* by:
    + Remove columns '<i>totaltaxvalue</i>', '<i>buildvalue</i>' or '<i>landvalue</i>' from the training and testing set and also '<i>mypointer</i>' from the training set
    + Removing features that have more than 40% of missing data in the training set 
    + After that, removing observations that have missing data
3. Create *dummy variables* for relevant categorical features
4. *Build* your model and test it on the same input data
5. Assess expected accuracy using *cross-validation*
6. Report which variable impacts more on results 

You may want to iterate to refine some of these steps once you get performance results in step 5.


## Main criteria for grading
From more to less important (the weighting of these components will vary between the in-class and extended projects):
+ Code runs
+ Parcel value prediction made
+ Accuracy of predictions for test properties is calculated (kaggle)
+ Linear Model, Ridge and LASSO have been used
+ Accuracy itself
+ Data preparation
+ Hyperparameter optimization (alphas)
+ Code is combined with neat and understandable commentary, with some titles and comments (demonstrate you have understood the methods and the outputs produced)
+ Improved methods from what we discussed in class (properly explained/justified)

###Importing Data and packages

In [176]:
import pandas as pd
from matplotlib import pyplot as plt
import plotly.express as px
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score 
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.linear_model import Lasso, LassoCV

# IMPORT THE DATA
from google.colab import files 
uploaded = files.upload()

Saving Copy of Regression_Supervised_Train.csv to Copy of Regression_Supervised_Train (8).csv
Saving Copy of Regression_Supervised_Test.csv to Copy of Regression_Supervised_Test (8).csv


# Train Data

In [177]:
# Name the data and separate data from the csv
df = pd.read_csv("Copy of Regression_Supervised_Train.csv", sep = ';' ,decimal = ',')

In [178]:
# Making a copy for editing the data frame
df_copy = df.copy()

In [179]:
# STEP 1. REMOVE COLUMNS: 'totaltaxvalue', 'buildvalue' or 'landvalue' and also 'mypointer' since they are highly
# correlatd with the value aimed to predict.
# We also remove 'permiterarea' since it has a 100% of NAs 

df_copy= df_copy.drop(["totaltaxvalue", "buildvalue","landvalue","mypointer", "perimeterarea"],axis='columns')


# STEP 2. Removing features that have more than 40% of missing data
# We remove features with more than 80% missing value for the sake of being able of imputing some variables and including them in our model

df_copy = df_copy[df_copy.columns[df_copy.isnull().mean() <= 0.8]] 


# STEP 3. Imputing the NAs of variables that can have a significant effect for the model

# 1. Qualitybuild: 
# We decided to impute the median from the ones that have a registered value (Because it is a scale on good and bad quality of the buildings, 
# so we thought ut made more sense to take the median than the mean)

df_copy['qualitybuild'] = df_copy['qualitybuild'].fillna(df_copy['qualitybuild'].median())

# 2. Numfullbath and Num34bath: We decided to drop numfullbath and num34bath since dropping just one of them does not make sense
# as they are complementary variables. Since num34bath has nearly 80% missing values, we drop them both: 

df_copy= df_copy.drop(["numfullbath"],axis='columns')
df_copy= df_copy.drop(["num34bath"],axis='columns')

# 3. Heatingtype : We impute this variable with the value equivalent to no having any kind of heating 

df_copy['heatingtype'] = df_copy['heatingtype'].fillna(13.0)

# 4. Airconditioner: Imputed with the "5" = None category, the asumption made here, is the same as the previous category:
# than the houses without information don't have airconditioner

df_copy['aircond'] = df_copy['aircond'].fillna(5.0)

# 5. Lotarea: We do the filling of the missing values with the computed mean of the variable

df_copy['lotarea'] = df_copy['lotarea'].fillna(df_copy['lotarea'].mean())

# 6. Unitnum: Decided to drop the variable since it wouldn't do much for the variability of the model, it is almost as adding a constant since
#  when doing the value count for this variable we get that most of the observations are respect category 1.

df_copy= df_copy.drop(["unitnum"],axis='columns')

# 7. Garagenum: We decided to impute with the median (to get the central value within the sample)

df_copy['garagenum'] = df_copy['garagenum'].fillna(df_copy['garagenum'].median())

# 8. Garage area: We imputed this variable with the mean of the given values for having a numerical representation of the variable

df_copy['garagearea'] = df_copy['garagearea'].fillna(df_copy['garagearea'].mean())

# 9. Poolnum: We assume that the observations with no value for this variables have no pool, 
# therefore, we impute it with the value 0, corresponding to having no pool

df_copy['poolnum'] = df_copy['poolnum'].fillna(0.0)

# 10. Finishedarea1st and FinishedareaEntry: We decided to drop both of them, since they have 75% of missing values
# and a high correlation with each other and with 'finishedarea'

df_copy= df_copy.drop(["finishedarea1st"],axis='columns')
df_copy= df_copy.drop(["finishedareaEntry"],axis='columns')

# 11. Countycode and Countycode2: They have 0.62 correlation but provide similar information so we decided to drop one of them

df_copy= df_copy.drop(["countycode2"],axis='columns')

# 12. Neighborhoodcode: We decided to drop the variable since it has a lot and use the regioncode(zipcode) as location information instead
df_copy= df_copy.drop(["neighborhoodcode"],axis='columns')

# 13. Numstories : we were going to relate them to the citycode, assuming that every city has a prototype for the houses' levels
# but it is not significant because 64% of the observations are NAs

df_copy = df_copy.drop(["numstories"],axis='columns')

# 14. Year: We are going to impute this variable with the median of the observed values

df_copy['year'] = df_copy['year'].fillna(df_copy['year'].median())

In [180]:
# Finally, we will drop missing values from the data frame at once, for not having a mismatch in between variables: 

df_copy = df_copy.dropna()

In [181]:
# STEP 4. Create dummy variables for relevant categorical features: 

df_copy = pd.get_dummies(data=df_copy, columns=['taxyear','heatingtype','aircond', 'qualitybuild','countycode','regioncode', 'citycode'],drop_first = False)

In [182]:
# Distribution of parcelvalue
# We elaborate a histogram to visualize the distribution of the variable parcelvalue
fig = px.histogram(data_frame = df_copy,x = 'parcelvalue',nbins = 20000,color_discrete_sequence=['indianred']).update_layout(
    title={"text": "Distribution of parcelvalue", "x": 0.5}, yaxis_title="Frecuency", bargap = 0.1)
fig.show()

#It is possible to see that the distribution it is positively skewed 

In [183]:
# We decided because the possitive skeweness ditribution use the logarithm of parcel value, so we could improve the fit of the model 
# by transforming the distribution of the dependant variable to a more normally-shaped bell curve.
df_copy['parcelvalue'] = np.log1p(df_copy['parcelvalue'])

In [184]:
#We elaborate a histogram to visualize the distribution of the variable 'logparcelvalue'
fig = px.histogram(data_frame = df_copy,x = 'parcelvalue',nbins = 100,color_discrete_sequence=['indianred']).update_layout(
    title={"text": "Distribution of log-parcelvalue", "x": 0.5}, xaxis_title = "Log-parcelvalue", yaxis_title="Frecuency", bargap = 0.1)
fig.show()
#Now it's possible to visualice that the distribution follows a normally-shaped bell curve

Once we have our train model cleaned and imputed, we are going to proceed with the application of the following models respectively: Linear Model, Ridge Model, and finally, Lasso Model. The aim is to asses the best prediction possible for the model, optimizing the coefficients and getting a good fit. 

## Linear Model

In [185]:
# Arrange data into features and target

X = df_copy.loc[:, ~df_copy.columns.isin(['lotid', 'parcelvalue'])]
y = df_copy.loc[:, ['parcelvalue']]

# Split data
seed = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = seed, train_size = .80)

# Create regressor
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Predict test data
y_pred = np.squeeze(regressor.predict(X_test))

#Assesing expected accuracy by MSE, R2 and R2 with Cross Validation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
scores = cross_val_score(regressor, X_train, y_train, scoring='r2', cv=10)

## Ridge and Lasso Models

### Standarizing

In [186]:
# Data standardization helps improve the quality of the data by transforming and standardizing it. 
# Standarizing: We will scale all variables except the target
target = 'parcelvalue'
id = 'lotid'

#Scale vars
vars_cols = [x for x in df_copy.columns if x != target and x != id]
scaler = StandardScaler().fit(df_copy[vars_cols])
scaled_features = scaler.transform(df_copy[vars_cols])
df_scaled = pd.DataFrame(scaled_features, index = df_copy.index, columns = df_copy[vars_cols].columns)

# First dataset: original
X_original = df_scaled
Y_original = df_copy[target]
X_original.shape

(21913, 564)

### Ridge Model

In [187]:
# We are going to run regressions with penalty. The ridge model is a method of estimating the coefficients of multiple-regression models in scenarios where the independent variables are highly correlated.
# For starting we define random alpha and run a preliminary ridge regression
alpha = 188

ridge_model = Ridge(alpha = alpha)
ridge_mse = -cross_val_score(estimator = ridge_model,
                             X = X_original,
                             y = Y_original,
                             cv = 5,
                             scoring = 'neg_mean_squared_error').mean()
ridge_r2 = cross_val_score(estimator = ridge_model,
                           X = X_original,
                           y = Y_original,
                           cv = 5,
                           scoring = 'r2').mean()

In [188]:
# It's possible to improve the model finding the optimal value of alpha
# The alpha Alpha (α) is the penalty term that denotes the amount of shrinkage (or constraint) that will 
# be implemented in the equation. 
alphas = np.logspace(start = 0, stop = 5, num = 1000)

# Then we run a ridge model to find the optimal alpha value
ridge_model = RidgeCV(alphas = alphas, store_cv_values = True)
ridge_model = ridge_model.fit(X_original, Y_original)

# We obtain the value of the optimal alpha and print it
ridge_alpha_optimal = ridge_model.alpha_

print("Optimal alpha for ridge:", ridge_alpha_optimal)

Optimal alpha for ridge: 1239.0621569479156


In [189]:
# Therefore now we obtain the model quality indicators for the optimal value, mse and r2, using cross validation spliting the data in 5 parts.
ridge_model = Ridge(alpha = ridge_alpha_optimal)
ridge_mse = -cross_val_score(estimator = ridge_model,
                             X = X_original,
                             y = Y_original,
                             cv = 5,
                             scoring = 'neg_mean_squared_error').mean()
ridge_r2 = cross_val_score(estimator = ridge_model,
                           X = X_original,
                           y = Y_original,
                           cv = 5,
                           scoring = 'r2').mean()
# We will compare this values with the same of the other regression

### Lasso Model

In [190]:
# The Lasso model it is used over regression methods for a more accurate prediction. 
# This model uses shrinkageis where data values are shrunk towards a central point as the mean.
# As the same of the ridge model we need to define a value of alpha, so we start with a random value.
# And obtain the measures of fit of the model
alpha = 0.1

lasso_mse = -cross_val_score(estimator = lasso_model,
                             X = X_original,
                             y = Y_original,
                             cv = 5,
                             scoring = 'neg_mean_squared_error').mean()
lasso_r2 = cross_val_score(estimator = lasso_model,
                           X = X_original,
                           y = Y_original,
                           cv = 5,
                           scoring = 'r2').mean()

In [191]:
# Now we look for the optimal value of the parameter alpha of the model

alphas = np.arange(0.0001, 0.1, 0.0001)

# Run model to find the optimal value of alpha
lasso_model = LassoCV(alphas = alphas, cv = 5)
lasso_model = lasso_model.fit(X_original, Y_original)

# We obtain and print the value. 
lasso_alpha_optimal = lasso_model.alpha_

print("Optimal alpha for lasso:", lasso_alpha_optimal)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.2988312246516216, tolerance: 1.1313651456381002

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.677139393241305, tolerance: 1.1314218242631364

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.321570846129362, tolerance: 1.1409213819525463

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.8457938784031285, tolerance: 1.1142

Optimal alpha for lasso: 0.0019000000000000002


In [192]:
# Therefore we obtain the measures of fit of the model with the optimal value of alpha. 
lasso_model = Lasso(alpha = lasso_alpha_optimal)
lasso_mse = -cross_val_score(estimator = lasso_model,
                             X = X_original,
                             y = Y_original,
                             cv = 5,
                             scoring = 'neg_mean_squared_error').mean()
lasso_r2 = cross_val_score(estimator = lasso_model,
                           X = X_original,
                           y = Y_original,
                           cv = 5,
                           scoring = 'r2').mean()

### Model comparisson

In [ ]:
print(f'Mean squared error: {mse:.2f}')
print("Optimal Lasso CV mean MSE:", lasso_mse)
print("Optimal Ridge CV mean MSE:", ridge_mse)

print('cross validation r-Squared:', scores.mean())
print("Optimal Lasso mean R2:", lasso_r2)
print("Optimal Ridge mean R2:", ridge_r2)

Mean squared error: 0.31
Optimal Lasso CV mean MSE: 0.3068249040459431
Optimal Ridge CV mean MSE: 0.3099360314192031
cross validation r-Squared: 0.5155465262643213
Optimal Lasso mean R2: 0.5234288622512205
Optimal Ridge mean R2: 0.518604158095546


### Regression Coefficients

Comparing the MSE values we choose to use Ridge coefficients. (Since we have very close R2 we look at the MSE to define)

In [193]:
ridge_model = Ridge(alpha = ridge_alpha_optimal)
ridge_model.fit(X_original, Y_original)

# Creating  table with coefficients
ridge_coefficients = pd.DataFrame(ridge_model.coef_, 
                                  X_original.columns, 
                                  columns = ['coef']).sort_values(by = 'coef', ascending = False)

ridge_coefficients

,coef
finishedarea,0.240853
numbath,0.111261
numfireplace,0.059157
numbedroom,0.055686
regioncode_96050.0,0.055049
...,...
regioncode_96411.0,-0.029075
qualitybuild_6.0,-0.029272
regioncode_97104.0,-0.031679
lotarea,-0.037715


# Test Data

At this point, having made the estimations for the Train data, we will proceed to compare it with the given Test data

In [194]:
# Name the data and separate data from the csv
df2 = pd.read_csv("Copy of Regression_Supervised_Test.csv", sep = ';' ,decimal = ',')

In [195]:
# Making a copy for editing the data frame
df2_copy = df2.copy()

In [196]:
# STEP 1. REMOVE COLUMNS: 'totaltaxvalue', 'buildvalue' or 'landvalue' and also 'mypointer' since they are highly
# correlatd with the value aimed to predict.
# We also remove 'permiterarea' since it has a 100% of NAs 

df2_copy= df2_copy.drop(["totaltaxvalue", "buildvalue","landvalue", "perimeterarea"],axis='columns')

# STEP 2. Removing features that have more than 40% of missing data
# We remove features with more than 80% missing value for the sake of being able of imputing some variables and including them in our model

df2_copy = df2_copy[df2_copy.columns[df2_copy.isnull().mean() <= 0.8]] 

# STEP 3. Imputing the NAs of variables that can have a significant effect for the model
# We will do the same imputations for the test data made in the train data:

# 1. Qualitybuild 

df2_copy['qualitybuild'] = df2_copy['qualitybuild'].fillna(df2_copy['qualitybuild'].median())

# 2. Numfullbath and Num34bath 

df2_copy= df2_copy.drop(["numfullbath"],axis='columns')
df2_copy= df2_copy.drop(["num34bath"],axis='columns')

# 3. Heatingtype 

df2_copy['heatingtype'] = df2_copy['heatingtype'].fillna(13.0)

# 4. Airconditioner 

df2_copy['aircond'] = df2_copy['aircond'].fillna(5.0)

# 5. Lotarea 

df2_copy['lotarea'] = df2_copy['lotarea'].fillna(df2_copy['lotarea'].mean())

# 6. Unitnum

df2_copy= df2_copy.drop(["unitnum"],axis='columns')

# 7. Garagenum

df2_copy['garagenum'] = df2_copy['garagenum'].fillna(df2_copy['garagenum'].median())

# 8. Garage area

df2_copy['garagearea'] = df2_copy['garagearea'].fillna(df2_copy['garagearea'].mean())

# 9. Poolnum

df2_copy['poolnum'] = df2_copy['poolnum'].fillna(0.0)

# 10. Finishedarea1st and FinishedareaEntry 

df2_copy= df2_copy.drop(["finishedarea1st"],axis='columns')
df2_copy= df2_copy.drop(["finishedareaEntry"],axis='columns')

# 11. Countycode and Countycode2 

df2_copy= df2_copy.drop(["countycode2"],axis='columns')

# 12. Neighborhoodcode 

df2_copy= df2_copy.drop(["neighborhoodcode"],axis='columns')

# 13. Numstories

df2_copy= df2_copy.drop(["numstories"],axis='columns')

# 14. Year 

df2_copy['year'] = df2_copy['year'].fillna(df2_copy['year'].mean())

In [197]:
# STEP 4. Create dummy variables for relevant categorical features

df2_copy = pd.get_dummies(data=df2_copy, columns=['taxyear','heatingtype','aircond', 'qualitybuild','countycode','regioncode', 'citycode'],drop_first = False)
df2_copy.head()

,lotid,numbath,numbedroom,finishedarea,numfireplace,garagenum,garagearea,latitude,longitude,lotarea,...,citycode_51239,citycode_51617,citycode_52650,citycode_53571,citycode_54299,citycode_54311,citycode_54352,citycode_54722,citycode_55753,citycode_396054
0,10987932,1,3,1024,0,2.0,509.310811,34254627,-118406678,8126.000000,...,0,0,0,0,0,0,0,0,0,0
1,17171405,2,4,1974,0,1.0,375.000000,34150927,-119215574,2450.000000,...,0,0,0,0,0,0,0,0,0,0
2,10790981,3,3,1635,0,2.0,509.310811,34184000,-118598000,224297.000000,...,0,0,0,0,0,0,0,0,0,0
3,14531417,2,2,1097,0,1.0,218.000000,33494800,-117670000,36082.477157,...,0,0,0,0,0,0,0,0,0,0
4,11072694,3,3,2844,0,2.0,509.310811,34262146,-118561422,16452.000000,...,0,0,0,0,0,0,0,0,0,0


In [198]:
# Arrange data into features to get y_predic
#X = df2_copy.loc[:, ~df2_copy.columns.isin(['lotid', 'mypointer'])]


# Standarizing: We will scale all variables except the target
mypointer = 'mypointer'
id = 'lotid'

#Scale vars
vars_cols = [x for x in df2_copy.columns if x != mypointer and x != id]
scaler = StandardScaler().fit(df2_copy[vars_cols])
scaled_features = scaler.transform(df2_copy[vars_cols])
df2_scaled = pd.DataFrame(scaled_features, index = df2_copy.index, columns = df2_copy[vars_cols].columns)

# First dataset: original
X = df2_scaled

In [199]:
# Get missing columns in the training test
list_a = X_train.columns
list_b = X.columns

def get_difference(list_a, list_b):
    return set(list_a)-set(list_b)

non_match = list(get_difference(list_a, list_b))

# Add a missing column in test set with default value equal to 0
for c in non_match:
    X[c] = 0
# Ensure the order of column in the test set is in the same order than in train set 
X = X[X_train.columns]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`



In [ ]:
# Predict test data
Y = np.squeeze(ridge_model.predict(X))

### Kaggle Predictions Submissions

Once you have produced testset predictions you can submit these to <i> kaggle </i> in order to see how your model performs. 

The following code provides an example of generating a <i> .csv </i> file to submit to kaggle
1) create a pandas dataframe with two columns, one with the test set "lotid"'s and the other with your predicted "parcelvalue" for that observation

2) use the <i> .to_csv </i> pandas method to create a csv file. The <i> index = False </i> is important to ensure the <i> .csv </i> is in the format kaggle expects 

In [200]:
# Step 8: Produce .csv for kaggle testing 
test_predictions_submit = pd.DataFrame({"lotid": df2_copy.lotid, "parcelvalue": np.log(Y)})
test_predictions_submit.to_csv("test_predictions_submit.csv", index = False)
test_predictions_submit

,lotid,parcelvalue
0,10987932,11.920750
1,17171405,12.672702
2,10790981,12.840465
3,14531417,12.822656
4,11072694,13.590015
...,...,...
195,10766230,12.878258
196,14412894,13.170967
197,11039638,12.633605
198,17162761,12.583210
